In [1]:
import tensorflow as tf
# import tensorflow as tf
print(tf.__version__)
import torch
torch.cuda.is_available()

2.4.0


True

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
torch.__version__

Using TensorFlow backend.


'1.7.1+cu110'

In [3]:
MAX_LEN = 64
bs = 16 # batch_size

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

import re
import nltk
import pickle

dic_list = []
train_dic = {}
mask_counter = 0

def get_dic(src_words, tar_words):
    i = 0
    j = 0
    while i < len(src_words):
        if src_words[i] == tar_words[j]:
            i+=1
            j+=1
        else:
            p = i+1
            q = j+1
            
            while p<len(src_words):
                while q<len(tar_words) and tar_words[q]!=src_words[p]:
                    q += 1
                if q==len(tar_words):
                    p = p + 1
                    q=j+1
                else:
                    break
            print(src_words[i:p],tar_words[j:q])
            i=p
            j=q
    

def get_mask(src_words, tar_words, is_train=False):
    temp = []
    tar_ptr = 0
    for i, word in enumerate(src_words):
        if i+1 < len(src_words) and src_words[i+1] in tar_words[tar_ptr:]:
            j = tar_words.index(src_words[i+1], tar_ptr)+1
        else:
            j = len(tar_words)
        if word not in tar_words[tar_ptr:j]:
            temp.append(1)
            if is_train and word==word.upper():
                global mask_counter
                mask_counter +=1
        else:
            temp.append(0)
            tar_ptr = tar_words.index(word, tar_ptr) + 1
    return temp


def get_train_data():
    src = []
    tar = []
    txt = ''
    try:
        txt += open('./data_tmp/train(12809).txt', 'r').read()
    except:
        txt += open('./data_tmp/train(12809).txt', 'r', encoding='utf-8').read()
  
    txt = txt.split('\n\n')
    mask_new = []
    for para in tqdm(txt):
        sentences = para.split('\n')
        if len(sentences) < 2:
            continue
        for sid, sentence in enumerate(sentences[0:2]):
            if sid == 0:
                src.append(sentence)
            else:
                tar.append(sentence)

    for i in range(len(src)):
        src_sentence = src[i]
        tar_sentence = tar[i]
        src_words = nltk.word_tokenize(src_sentence)
        tar_words = nltk.word_tokenize(tar_sentence)
        mask_new.append(get_mask(src_words, tar_words,True))
        if i<10:
            get_dic(src_words,tar_words)
    
    src_new = []
    for sentence in src:
        temp = []
        words = nltk.word_tokenize(sentence)
        src_new.append(words)
    
    return src_new, mask_new

def get_test_data():
    txt = open('./data_tmp/test(2030).txt', 'r').read()
    txt = txt.lower()
    txt = txt.split('\n\n')
    mask = []
    src_new = []
    for para in tqdm(txt):
        sentences = para.split('\n')
        masks = []
        
        src_sentence = ''
        
        if len(sentences) < 2 or len(sentences[0]) < 3 or len(sentences[1]) < 3:
            continue
        for sid, sentence in enumerate(sentences):
            if sid == 0:
                src_sentence = sentence
                words = nltk.word_tokenize(sentence)
                src_new.append(words)
                
            elif sid == 1:
                cudic = {}
                sentence = sentence[2:]
                text = re.sub('\[[^\[\]]*\]', '', sentence)
                pairs = re.findall('[^\[\] ]+\[[^\[\]]+\]', sentence)
                for pair in pairs:
                    pair = re.split('[\[\]]', pair)
                    cudic[pair[0]] = pair[1]
                    dic_list.append(pair)
                words = nltk.word_tokenize(text)
                for wid, word in enumerate(words):
                    if word in cudic.keys():
                        words[wid] = cudic[word]
                new_text = ''
                for word in words:
                    new_text += word
                    new_text += ' '
                masks=get_mask(nltk.word_tokenize(src_sentence), nltk.word_tokenize(new_text))

        mask.append(masks)
    return src_new, mask

In [5]:
train_sentences, train_labels = get_train_data()
test_sentences, test_labels = get_test_data()

100%|██████████| 12801/12801 [00:00<00:00, 759069.82it/s]


['ulcers'] ['break', 'in', 'skin']
['Hco3'] ['Bicarbonate']
['wbc'] ['white', 'blood', 'cells']
['tox', 'screens'] ['toxicology', 'test']
['Known', 'lastname'] ['Patient']
['VPS'] ['shunt']
['CXR'] ['chest', 'x-ray']
['pneumonia'] ['lung', 'infection']
['gravida'] ['number', 'of', 'pregnancies']
['para'] ['number', 'of', 'completed', 'pregnancies']
['CT-Torso'] ['Torso', 'Imaging']


100%|██████████| 2031/2031 [00:01<00:00, 1892.22it/s]


In [6]:
print(mask_counter)

11086


In [7]:
print(train_sentences[:5])
print(train_labels[:5])
print(test_sentences[:5])
print(test_labels[:5])

[['Stage', '2', 'ulcers', ':', 'On', 'sacrum', 'and', 'scrotum', '.'], ['Labs', 'were', 'significant', 'for', 'd-dimer', '6218', ',', 'troponin', '0.05', ',', 'creatinine', '1.4', ',', 'Hco3', '20', 'and', 'wbc', '11.3', '.'], ['They', 'drove', 'down', 'from', 'Country', '6607', 'to', 'Hospital1', '18', 'Emergency', 'Department', 'for', 'evaluation', '.'], ['Serum', 'tox', 'screens', 'were', 'negative', 'for', 'benzodiazepenes', ',', 'barbituates', 'and', 'tricyclic', 'antidepressants', '.'], ['Known', 'lastname', 'was', 'admitted', 'to', 'the', 'Neurosurgery', 'service', 'after', 'a', 'study', 'revealed', 'a', 'break', 'in', 'her', 'VPS', 'catheter', 'at', 'the', 'level', 'of', 'her', 'neck', '.']]
[[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]
[['weaned', 'off', 'vent', 'to', 'cpap',

In [8]:
tag_values = [0, 1]
tag_values.append("PADT")
tag2idx = {t: i for i, t in enumerate(tag_values)}

tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(train_sentences, train_labels)
]

tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PADT"], padding="post",
                     dtype="long", truncating="post")
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]
pickle.dump(input_ids, open("./data_tmp/train_input_ids.pkl",'wb'))
pickle.dump(tags, open("./data_tmp/train_tags.pkl",'wb'))
pickle.dump(attention_masks, open("./data_tmp/train_attention_masks.pkl",'wb'))


In [10]:
import numpy as np
print(np.max(input_ids),np.min(input_ids))
print(np.max(tags),np.min(tags))
print(np.max(attention_masks),np.min(attention_masks))


28117 0
2 0
1.0 0.0


In [9]:

tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(test_sentences, test_labels)
]
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PADT"], padding="post",
                     dtype="long", truncating="post")
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

pickle.dump(input_ids, open("./data_tmp/test_input_ids.pkl",'wb'))
pickle.dump(tags, open("./data_tmp/test_tags.pkl",'wb'))
pickle.dump(attention_masks, open("./data_tmp/test_attention_masks.pkl",'wb'))



In [10]:
test_dic = {}
for pair in dic_list:
    if pair[0] in test_dic.keys():
        test_dic[pair[0]].add(pair[1])
    else:
        test_dic[pair[0]]=set([pair[1]])
        
print(test_dic)

{'cpap': {'continuous positive airway pressure', 'continuous positive airway pressure therapy'}, 'pea': {'pulseless electrical activity'}, 'mri': {'magnetic resonance imaging'}, 'pt': {'prothrombin time', 'patient', 'physical therapy', 'physical therapist', 'physical therapy '}, 'yo': {'years old', 'year old'}, 'sss': {'sick sinus syndrome'}, 's/p': {'condition after', 'status post'}, 'sdh': {'subdural hematoma'}, 'pod': {'postoperative day', 'postoperative days'}, 'inr': {'international normalized ratio'}, 'po': {'medication is taken by mouth', 'by mouth', 'oral'}, 'paf': {'paroxysmal atrial fibrillation'}, 'pvd': {'peripheral vascular disease'}, 'hld': {'hyperlipidemia'}, 'htn': {'hypertension'}, 'osa': {'obstructive sleep apnea'}, 'pcp': {'primary care provider', 'pneumocystis pneumonia', 'pneumocystis carinii pneumonia ', 'pneumocystis carinii pneumonia'}, 'ccu': {'critical care unit', 'coronary care unit'}, 'nad': {'no abnormality detected'}, 'sob': {'shortness of breathing', 'sho

In [11]:
with open('./tran_data/medical-dictionary-v2.pkl','rb') as f:
    abbrs = pickle.load(f)

In [12]:
abbrs['peg']

['Percutaneous Endoscopy Gastrostomy']

In [13]:
uncase_abbrs = {}
for key in abbrs.keys():
    abbr = key.lower()
    if abbr in uncase_abbrs.keys():
        uncase_abbrs[abbr].extend(abbrs[key])
    else:
        uncase_abbrs[abbr]=abbrs[key]
        
for key in uncase_abbrs.keys():
    uncase_abbrs[key] = [u.lower() for u in uncase_abbrs[key]]
        

In [14]:
cnt = 0
num = 0
tot = 0
for key in test_dic.keys():
    if key in uncase_abbrs.keys():
        cnt += 1
        flag = False
        for test_expr in test_dic[key]:
            for u in uncase_abbrs[key]:
                if test_expr.strip() in u:
                    num+=1
                    flag = True
#                     print(test_expr,':',u)
                    break
            tot +=1
            if not flag:
                print(key,':',test_expr)
    else:
        print(key)

print(len(test_dic.keys()),cnt, cnt/len(test_dic.keys()))
print(tot, num, num/tot)

pt : prothrombin time
s/p : condition after
po : medication is taken by mouth
sob : shortness of breathing
ed : emergnecy department
pna : pneumonia
pna : postnatal age ; pneumonia
pna : pulmonary nodular amyloidosis
cdiff
hrs : heart rate
ct : computed tomography
dvt : deep vein thrombosis
osh : highly active antiretroviral therapy
d/c'ed
vna : visiting nursing association
bka : below knee amputation
r/o : removal of
ppi : proton-pump inhibitors
icd : intercostal drain
icd : implantable cardioverter-defibrillator
icd : implantable cardioverter-defibrillators
/dni
nchct : non-contrast helical computerized tomography
copd : obstructive lung disease
rbcs
59yo
acs/nstemi
sxs : symptoms
p/f
maps : mean arterial pressure
flagyl/po
pod1 : postoperative days 1
avms
i : finds
i : intravenous
/ercp
ugib
gpc : gram-positive cocci
gpc : gram positive cocci
lfts
d/c : iscontinued
ra : refractory anemia
ra : room air
ra : right atrium
vicu
re-egd
bms : bowel movements
bms : basal metabolic rate
n :

In [15]:
print(uncase_abbrs['icd'])

['implantable cardioverter defibrillator']


In [16]:
print('physical therapy' in 'physical therapy')

True
